In [ ]:

# get_option_md_client = OptionHistoricalDataClient(self.api_key, self.api_secret)
# 


In [ ]:
req = OptionLatestTradeRequest(symbol_or_symbols=symbol_or_symbols)
option_md_client.get_option_latest_trade(req)

In [ ]:
ap.get_latest_trade(symbol_or_symbols='SPY240920C00325000',asset='option')

In [ ]:
!pip install alpaca-py

In [ ]:
import alpaca

In [1]:
import pandas as pd
import json
from TAI.source import Alpaca


ap = Alpaca()

# Fetch the option chain data
df_chain = ap.get_option_chain(
    underlying_symbol='SPY',
    feed=None,
    type=None,
    strike_price_gte=None,
    strike_price_lte=None,
    expiration_date='2024-10-18',
    expiration_date_gte=None,
    expiration_date_lte=None,
    root_symbol='SPY',
    raw=False,
    chain_table=True
)
df_chain

,Call Bid,Call Ask,Call Volume,Call Open Interest,Call Delta,Call Gamma,Call Vega,Call Theta,Strike,Put Bid,Put Ask,Put Volume,Put Open Interest,Put Delta,Put Gamma,Put Vega,Put Theta
0,307.38,312.45,10.0,714,NaN,NaN,NaN,NaN,260.0,0.03,0.04,923.0,2130,NaN,NaN,NaN,NaN
1,303.98,304.92,10.0,3,NaN,NaN,NaN,NaN,265.0,0.01,0.04,875.0,4605,NaN,NaN,NaN,NaN
2,295.67,302.81,10.0,52,NaN,NaN,NaN,NaN,270.0,0.03,0.04,822.0,5920,NaN,NaN,NaN,NaN
3,297.02,300.56,10.0,150,NaN,NaN,NaN,NaN,275.0,0.03,0.04,768.0,3981,NaN,NaN,NaN,NaN
4,293.20,294.12,10.0,1,NaN,NaN,NaN,NaN,280.0,0.01,0.02,713.0,2965,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0.03,0.04,273.0,1811,0.0038,0.0004,0.0149,-0.0093,660.0,88.04,89.67,10.0,None,-0.9184,0.0027,0.1967,-0.1403
252,0.03,0.04,811.0,1466,0.0037,0.0003,0.0143,-0.0094,665.0,91.60,92.22,10.0,None,-0.9937,0.0005,0.0232,0.0725
253,0.03,0.04,866.0,3085,0.0035,0.0003,0.0138,-0.0094,670.0,100.40,101.01,10.0,None,-0.8831,0.0028,0.2561,-0.2880
254,0.03,0.04,827.0,5497,0.0034,0.0003,0.0134,-0.0095,675.0,104.68,105.29,10.0,None,-0.8995,0.0025,0.2296,-0.2413


In [4]:
import pandas as pd
import json
from TAI.source import Alpaca

def get_option_chain_json(underlying_symbol, expiration_date):
    ap = Alpaca()
    
    # Fetch the option chain data
    df_chain = ap.get_option_chain(
        underlying_symbol=underlying_symbol,
        feed=None,
        type=None,
        strike_price_gte=None,
        strike_price_lte=None,
        expiration_date=expiration_date,
        expiration_date_gte=None,
        expiration_date_lte=None,
        root_symbol=underlying_symbol,
        raw=False,
        chain_table=True
    )

    data = {
        underlying_symbol: {
            "expiration_dates": {
                expiration_date: {
                    "option_chain": []
                }
            }
        }
    }

    # Helper function to convert NaN to None (which becomes null in JSON)
    def convert_nan(value):
        if pd.isnull(value):
            return None
        else:
            return value

    # Define the columns for calls and puts
    call_columns = ['Call Bid', 'Call Ask', 'Call Volume', 'Call Open Interest',
                    'Call Delta', 'Call Gamma', 'Call Vega', 'Call Theta']
    call_json_keys = ['bid', 'ask', 'volume', 'open_interest',
                      'delta', 'gamma', 'vega', 'theta']

    put_columns = ['Put Bid', 'Put Ask', 'Put Volume', 'Put Open Interest',
                   'Put Delta', 'Put Gamma', 'Put Vega', 'Put Theta']
    put_json_keys = ['bid', 'ask', 'volume', 'open_interest',
                     'delta', 'gamma', 'vega', 'theta']

    # Iterate over each row in the DataFrame to build the JSON structure
    for _, row in df_chain.iterrows():
        call_data = {}
        for col, key in zip(call_columns, call_json_keys):
            call_data[key] = convert_nan(row[col])

        put_data = {}
        for col, key in zip(put_columns, put_json_keys):
            put_data[key] = convert_nan(row[col])

        option_entry = {
            'strike': convert_nan(row['Strike']),
            'call': call_data,
            'put': put_data
        }

        data[underlying_symbol]['expiration_dates'][expiration_date]['option_chain'].append(option_entry)

    # Convert the data to a JSON string
    json_string = json.dumps(data, indent=2)
    
    # Output the JSON string
    print(json_string)

# Example usage:
get_option_chain_json('MSFT', '2024-10-18')


{
  "MSFT": {
    "expiration_dates": {
      "2024-10-18": {
        "option_chain": [
          {
            "strike": 210.0,
            "call": {
              "bid": 216.02,
              "ask": 216.96,
              "volume": 3.0,
              "open_interest": "29",
              "delta": null,
              "gamma": null,
              "vega": null,
              "theta": null
            },
            "put": {
              "bid": 0.0,
              "ask": 0.01,
              "volume": 2.0,
              "open_interest": "376",
              "delta": null,
              "gamma": null,
              "vega": null,
              "theta": null
            }
          },
          {
            "strike": 220.0,
            "call": {
              "bid": 210.94,
              "ask": 211.64,
              "volume": 3.0,
              "open_interest": "5",
              "delta": null,
              "gamma": null,
              "vega": null,
              "theta": null
            },

In [ ]:
from TAI.source import Alpaca
ap = Alpaca()
import pandas as pd
import json


df_chain = ap.get_option_chain(underlying_symbol='SPY',
                    feed=None,
                    type=None,
                    strike_price_gte=None, #550.0,
                    strike_price_lte=None, #560.0,
                    expiration_date='2024-10-18',
                    expiration_date_gte=None,
                    expiration_date_lte=None,
                    root_symbol='SPY',
                    raw=False,
                    chain_table=True)

def dataframe_to_json(df, symbol, expiration_date):
    data = {
        symbol: {
            "expiration_dates": {
                expiration_date: {
                    "option_chain": []
                }
            }
        }
    }

    # Helper function to convert NaN to None
    def convert_nan(value):
        if pd.isnull(value):
            return None
        else:
            return value

    call_columns = ['Call Bid', 'Call Ask', 'Call Volume', 'Call Open Interest', 'Call Delta', 'Call Gamma', 'Call Vega', 'Call Theta']
    call_json_keys = ['bid', 'ask', 'volume', 'open_interest', 'delta', 'gamma', 'vega', 'theta']

    put_columns = ['Put Bid', 'Put Ask', 'Put Volume', 'Put Open Interest', 'Put Delta', 'Put Gamma', 'Put Vega', 'Put Theta']
    put_json_keys = ['bid', 'ask', 'volume', 'open_interest', 'delta', 'gamma', 'vega', 'theta']

    for _, row in df.iterrows():
        call_data = {}
        for col, key in zip(call_columns, call_json_keys):
            call_data[key] = convert_nan(row[col])

        put_data = {}
        for col, key in zip(put_columns, put_json_keys):
            put_data[key] = convert_nan(row[col])

        option_entry = {
            'strike': convert_nan(row['Strike']),
            'call': call_data,
            'put': put_data
        }

        data[symbol]['expiration_dates'][expiration_date]['option_chain'].append(option_entry)

    return data

# Example usage:
# Assuming 'df' is your DataFrame, 'SPY' is the symbol, and '2024-10-18' is the expiration date.

# df = pd.read_csv('your_dataframe.csv')  # Replace with your DataFrame loading method
symbol = 'SPY'
expiration_date = '2024-10-18'

json_data = dataframe_to_json(df_chain, symbol, expiration_date)

# Convert the data to a JSON string if needed
json_string = json.dumps(json_data, indent=2)

# Print or save the JSON string
print(json_string)




# [['strike_price','ask_price','bid_price','type']]


# df_chain.loc[df_chain['strike_price']==573.0]a

In [ ]:
df_chain.root_symbol

In [ ]:
df_chain.columns

In [ ]:
type(df_chain)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from TAI.source import Alpaca

class OptionBet:
    def __init__(self, ticker, expiry_date):
        self.ap = Alpaca()
    
        self.ticker = ticker
        self.expiry_date = expiry_date
        
        self.today = date.today()
        self.today_str = self.today.strftime("%Y-%m-%d")
        self.df_raw = self.ap.get_stock_historical(symbol_or_symbols=self.ticker, 
                                                lookback_period=720, 
                                                timeframe='Day', 
                                                end=None, 
                                                currency='USD', 
                                                limit=None, 
                                                adjustment='all', 
                                                feed=None, 
                                                asof=None, 
                                                sort='asc', 
                                                raw=False)
        
        self.df = self.df_raw[['timestamp','open','high','low','close','volume']].set_index('timestamp')
        self.current_price = self.ap.get_latest_trade(self.ticker).get(self.ticker).price
        self.open=self.df.tail(1)['open'][0]

    def nearest_value(self, input_list, find_value):
        difference = lambda input_list: abs(input_list - find_value)
        return min(input_list, key=difference)

    def weekdays_calculator(self, end_str):
        end = datetime.strptime(end_str, '%Y-%m-%d').date()
        return np.busday_count(self.today, end)

    def perc_change(self, df, horizon):
        df['return_perc'] = df.pct_change(periods=horizon - 1, fill_method='ffill').round(decimals=4)['close']
        return df.dropna()

    def describe_perc_change(self):
        horizon = self.weekdays_calculator(self.expiry_date)
        df_perc_change = self.perc_change(self.df, horizon)[['close', 'return_perc']]
        select_price = self.current_price
        describe = df_perc_change.describe(percentiles=[.001, .01, .05, .1, .15, .25, .5, .75, .85, .9, .95, .99, .999])
        describe['current_price'] = select_price
        describe['projected_price'] = select_price * (1 + describe['return_perc'])
        describe['percentage_change'] = describe['return_perc'] * 100
        describe['chose_strike'] = describe['projected_price'].astype(int)
        return describe[['current_price', 'percentage_change', 'projected_price', 'chose_strike']]


# Example usage
if __name__ == "__main__":
    ticker = 'SPY'
    chose_expiry_date = '2024-09-20'
    option_bet = OptionBet(ticker, chose_expiry_date)
    describe_df = option_bet.describe_perc_change()
    print(describe_df)
